In [33]:
import pandas as pd
from tensorflow import keras

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
from os import path
import pickle

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
DATA_DIR = "../../data/"

In [5]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [6]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [7]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [8]:
wandb.init(project=f"TrainFrozen1", entity="kvetab")

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [9]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
valid_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [10]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
valid_data["seq"] = valid_data["heavy"] + valid_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [11]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True),
    WandbCallback()
]
seq_len = 512

In [37]:
encoded_train_set, encoded_valid_set = encode_train_and_valid_sets(
    train_data['seq'], 
    train_data['Y'], 
    valid_data['seq'], 
    valid_data['Y'], 
    input_encoder, 
    OUTPUT_SPEC, 
    seq_len
)

[2022_02_02-16:12:17] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_02_02-16:12:17] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.


In [13]:
train_X, train_Y, train_sample_weigths = encoded_train_set

In [14]:
model = model_generator.create_model(seq_len=512, freeze_pretrained_layers=True)

2022-02-02 15:47:03.869304: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-02 15:47:04.505672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [15]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-2

In [16]:
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num,
      "batch_size": batch_size
    }

In [17]:
model.optimizer.lr = learning_rate

In [18]:
model.fit(
    x=train_X,
    y=train_Y,
    batch_size=batch_size,
    epochs=epoch_num,
    callbacks=training_callbacks,
    validation_data=encoded_valid_set
)


2022-02-02 15:47:25.340920: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-02-02 15:47:33.583853: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


11/11 [==============================] - 12s 298ms/step - loss: 0.7153 - val_loss: 0.5517


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/50
11/11 [==============================] - 1s 103ms/step - loss: 0.5146 - val_loss: 0.5106
Epoch 3/50
11/11 [==============================] - 1s 104ms/step - loss: 0.4646 - val_loss: 0.4768
Epoch 4/50
11/11 [==============================] - 1s 103ms/step - loss: 0.4534 - val_loss: 0.4688
Epoch 5/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4499 - val_loss: 0.4532
Epoch 6/50
11/11 [==============================] - 1s 105ms/step - loss: 0.4229 - val_loss: 0.4990

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 7/50
11/11 [==============================] - 1s 105ms/step - loss: 0.4460 - val_loss: 0.4396
Epoch 8/50
11/11 [==============================] - 1s 103ms/step - loss: 0.4303 - val_loss: 0.4460

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 9/50
11/11 [==============================] - 1s 103ms/step - loss: 0.4244 - val_loss: 0.4378
Epoch 10/50
11/11 [===================

In [19]:
model_generator.update_state(model)

In [20]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [18]:
import tensorflow as tf
print(tf. __version__)

2.6.2


In [22]:
def slice_arrays(arrays, slicing):
    if isinstance(arrays, list) or isinstance(arrays, tuple):
        return [array[slicing] for array in arrays]
    else:
        return arrays[slicing]

In [25]:
model_generator.dummy_epoch = (slice_arrays(train_X, slice(0, 1)), slice_arrays(train_Y, slice(0, 1)))
model = model_generator.create_model(seq_len=512, freeze_pretrained_layers=False)

[2022_02_02-15:56:35] Incompatible number of optimizer weights - will not initialize them.


In [28]:
wandb.init(project=f"TrainFrozen1", entity="kvetab")
epoch_num = 20
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num,
      "batch_size": batch_size
    }
model.optimizer.lr = learning_rate

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,████▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▃▅▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,48
best_val_loss,0.42643
epoch,49
loss,0.4096
lr,0.0
val_loss,0.42643


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [29]:
model.fit(
    x=train_X,
    y=train_Y,
    batch_size=batch_size,
    epochs=epoch_num,
    callbacks=training_callbacks,
    validation_data=encoded_valid_set
)

Epoch 1/20
11/11 [==============================] - 3s 258ms/step - loss: 0.4042 - val_loss: 0.4228
Epoch 2/20
11/11 [==============================] - 3s 253ms/step - loss: 0.3936 - val_loss: 0.4157
Epoch 3/20
11/11 [==============================] - 3s 254ms/step - loss: 0.3889 - val_loss: 0.4114
Epoch 4/20
11/11 [==============================] - 3s 259ms/step - loss: 0.3819 - val_loss: 0.4252

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 5/20
11/11 [==============================] - 3s 255ms/step - loss: 0.3598 - val_loss: 0.4039
Epoch 6/20
11/11 [==============================] - 3s 256ms/step - loss: 0.3540 - val_loss: 0.4015
Epoch 7/20
11/11 [==============================] - 3s 256ms/step - loss: 0.3467 - val_loss: 0.4008
Epoch 8/20
11/11 [==============================] - 3s 255ms/step - loss: 0.3376 - val_loss: 0.3988
Epoch 9/20
11/11 [==============================] - 3s 254ms/step - loss: 0.3461 - val_loss: 0.4008

Epoch 00009: Reduc

In [30]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-6
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num,
      "batch_size": batch_size
    }
model.optimizer.lr = learning_rate

In [31]:
model.fit(
    x=train_X,
    y=train_Y,
    batch_size=batch_size,
    epochs=epoch_num,
    callbacks=training_callbacks,
    validation_data=encoded_valid_set
)

Epoch 1/50
11/11 [==============================] - 3s 260ms/step - loss: 0.3360 - val_loss: 0.3947
Epoch 2/50
11/11 [==============================] - 3s 255ms/step - loss: 0.3289 - val_loss: 0.3951

Epoch 00002: ReduceLROnPlateau reducing learning rate to 2.499999993688107e-07.
Epoch 3/50
11/11 [==============================] - 3s 257ms/step - loss: 0.3237 - val_loss: 0.3951

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1e-07.
Epoch 4/50
11/11 [==============================] - 3s 255ms/step - loss: 0.3328 - val_loss: 0.3952


In [32]:
model.save(path.join(DATA_DIR, "protein_bertmanual_training_1_2022_02_02.pkl"))

2022-02-02 16:08:42.010264: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bertmanual_training_1_2022_02_02.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bertmanual_training_1_2022_02_02.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [36]:
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_02_02-16:12:09] Test set: Filtered out 0 of 119 (0.0%) records of lengths exceeding 510.


In [38]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [49]:
from sklearn.metrics import f1_score, confusion_matrix

In [40]:
y_pred = model.predict(test_X, batch_size=32)

In [47]:
y_pred_classes = (y_pred >= 0.5)

In [48]:
f1_score(test_Y, y_pred_classes)

0.5789473684210527

In [54]:
pd.DataFrame(confusion_matrix(test_Y, y_pred_classes, labels=[0, 1]), index=[0, 1], columns=[0, 1])

,0,1
0,92,4
1,12,11
